### Imports

In [2]:
from caveclient import CAVEclient
from intern import array
import pickle
import numpy as np
from agents import data_loader
from cloudvolume import CloudVolume
from membrane_detection import membranes
from agents.scripts import precompute_membrane_vectors, create_post_matrix, merge_paths, get_soma
import agents.sensor
from agents.run import run_agents
import aws.sqs as sqs
import sys
import time
import ast
import pandas as pd
import agents.scripts as scripts
from drive import drive
from finding_orphans import *
from math import sqrt
from tip_finding.tip_finding import tip_finder_decimation

### Helper functions

In [4]:
def find_euclidian_distance(proposed_endpoint, gt_endponts_array):
    diffs = gt_endponts_array - proposed_endpoint
    diffs_distance = np.sqrt(np.sum(np.square(diffs), axis=1))
    min_dist_ind = np.argmin(diffs_distance)
    min_dist = diffs_distance[min_dist_ind]
    return min_dist

In [5]:
def endpoint_generator(load, invalidation_d, humfrey_iters, decimation_factor):
    for idx, i in enumerate(load['seg_id']):
        if idx == 20:
            break
        try:
            t1, skel, mesh_obj = tip_finder_decimation(str(i))
            endpoints_dict[i] = t1
        except:
            print(f"\n\nSeg {i} returned error on get. Skipping.\n")
            pass
    load['endpoints'] = load['endpoints'].apply(
        lambda x: list(ast.literal_eval(x)))
    load['proposed_endpoints'] = load.seg_id.map(endpoints_dict)
    return load

In [6]:
def testing_metrics(load, threshold, run, output):
    for index, row in load.iterrows():
        if index == 20:
            break

        endpoint_array = np.array(row["endpoints"])
        proposed_endpoints_array = np.array(row["proposed_endpoints"])
        segID = row["seg_id"]

        #skip anything seg_id that isn't 'good'
        if row["comments"] != 'good':
            print("skipping \n")
            continue

        #if both say no endpoints, then it's correct
        elif (len(proposed_endpoints_array.shape) == 0 and len(endpoint_array.shape) == 0):
            out_df.at[index, 'precision'] = 1.0
            out_df.at[index, 'recall'] = 1.0
            out_df.at[index, 'f1'] = 1.0

            continue

        #if we propose no endpoints but there are endpoints, it's wrong
        elif (len(proposed_endpoints_array.shape) == 0 and len(endpoint_array.shape) > 0):
            out_df.at[index, 'precision'] = 0.0
            out_df.at[index, 'recall'] = 0.0
            out_df.at[index, 'f1'] = 0.0
            continue

        #if we propose endpoints but there are none, it's wrong
        elif proposed_endpoints_array.size > 0 and endpoint_array.size == 0:
            out_df.at[index, 'precision'] = 0.0
            out_df.at[index, 'recall'] = 0.0
            out_df.at[index, 'f1'] = 0.0
            continue

        #should skip for now if the endpoint array is size zero to avoid divide by zero error
        elif endpoint_array.size == 0:
            out_df.at[index, 'precision'] = null
            out_df.at[index, 'recall'] = null
            out_df.at[index, 'f1'] = null
            continue

        else:
            endpoint_ids = np.arange(0, len(endpoint_array))
            test_ids = np.arange(
                0, len(proposed_endpoints_array)) + len(proposed_endpoints_array)

            #get precision and recall at end of each iteration
            analysis = run_synapse_analysis(
                endpoint_array,
                np.array(endpoint_ids),
                proposed_endpoints_array,
                np.array(test_ids),
                threshold,
                iso_correction=10,
            )

            output.loc[len(output.index)] = [run, analysis.precision, analysis.recall, analysis.f1, segID, len(
                endpoint_array), endpoint_array, row["comments"], proposed_endpoints_array]
    return output

In [7]:
"""
Various utility classes and functions for Confirms.
"""
from copy import deepcopy

import numpy as np
import pandas as pd
from scipy import spatial
from scipy.optimize import linear_sum_assignment


def calculate_precision_recall(tp, fp, fn):
    """
    Calculate precision/recall from given true positives, false positives, and false negatives.
    Parameters
    ----------
    tp : int
        The number of true positives.
    fp : int
        The number of false positives.
    fn : int
        The number of false negatives.
    Returns
    -------
    precision : float
        The precision score.
    recall : float
        The recall score.
    """
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    return precision, recall


def calculate_f1(precision, recall):
    """
    Calculate the F1 score from precision/recall scores.
    Parameters
    ----------
    precision : float
        The precision score.
    recall : float
        The recall score.
    Returns
    -------
    f1 : float
        The F1 score.
    """
    return 2 * ((precision * recall) / (precision + recall))


def get_summary_metrics(array):
    """
    Calculate a number of summary metrics on an array of numbers.
    Parameters
    ----------
    array : array_like
        Array containing numbers who summary metrics is desired.
    Returns
    -------
    metrics : dict
        Dict containing the mean, median, max, min, range, standard deviation,
        and variance of the input array.
    """
    summary_object = {}
    array = np.array(array)

    summary_object["mean"] = np.mean(array)
    summary_object["median"] = np.median(array)
    summary_object["max"] = np.amax(array)
    summary_object["min"] = np.amin(array)
    summary_object["range"] = np.mean(array)
    summary_object["stddev"] = np.std(array)
    summary_object["variance"] = np.var(array)

    return summary_object


def munkres_assignment(workers, jobs):
    """
    Perform hungarian-munkres assignment.
    Parameters
    ----------
    workers : array_like
        Array containing the first set of points.
    jobs : array_like
        Array containing the second set of points.
    Returns
    -------
    cost_matrix : numpy.ndarray
        Matrix containing pairwise distances (the cost of assignment).
    row_ind : numpy.ndarray
        Row indices of cost_matrix for optimal assignment.
    col_ind : numpy.ndarray
        Column indices of cost_matrx for optimal assignment.
    """
    cost_matrix = spatial.distance.cdist(workers, jobs, "euclidean")
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    return cost_matrix, row_ind, col_ind


def make_isotropic(xyz, correction, dimen=2):
    """
    Correct anisotropy in a collection of x, y, z coordinates.
    This function performs a deepcopy of xyz before making the necessary modifications.
    Parameters
    ----------
    xyz : numpy.ndarray or pandas.DataFrame:
        The coordinate values.
    correction : float
        The value to correct anisotrophy.
    dimen : int
        Index of last dimension to which to apply the correction. Default is 2.
    Returns
    -------
    iso_xyz : numpy.ndarray or pandas.DataFrame
        An isotropic version of xyz.
    """

    if not isinstance(xyz, np.ndarray) and not isinstance(xyz, pd.DataFrame):
        xyz = np.asarray(xyz)

    shape = np.shape(xyz)
    isotropic_xyz = xyz.copy()
    size = shape[-1]
    if dimen >= size or dimen < 0:
        raise ValueError(
            "improper dimen value (valid: 0 through {})".format(size - 1))
    if isinstance(xyz, np.ndarray):
        isotropic_xyz[..., dimen] = isotropic_xyz[..., dimen] * correction
    else:
        isotropic_xyz.iloc[:, dimen] = isotropic_xyz.iloc[:,
                                                          dimen] * correction
    return isotropic_xyz

In [8]:
"""
Confirms synapse processing and analysis functions.
"""
from collections import namedtuple

import numpy as np
import pandas as pd
from scipy import spatial
from scipy.optimize import linear_sum_assignment

# from . import utils
# import utils

SynapseMetrics = namedtuple(
    "SynapseMetrics",
    ["precision", "recall", "f1", "tp_gt_ids", "tp_test_ids", "fp_ids", "fn_ids"],
)


def filter_synapse_id_core(volume, xyz, ids, box_radius_nm=2500):
    """
    Filter synapses to only return those within a central core.
    Parameters
    ----------
    volume : dict
        The volume to filter on.
    xyz : array_like
        Synapse xyz coordinates.
    ids : array_like
        Synapse hash (ids).
    box_radius_nm : int
        radius of cube, from volume core.
    Returns
    -------
        xyz : numpy.ndarray
            Synapse coordinates.
        ids : numpy.ndarray
            Synapse ids.
    """

    xyz_out = []
    id_out = []
    center = np.asarray(volume["center"], "float")
    base_resolution = np.asarray(volume["base_resolution"], "float")
    annotation_resolution = np.asarray(volume["resolution"], "float")

    pad_vx = box_radius_nm / base_resolution[0] / (2 ** annotation_resolution)
    pad_vy = box_radius_nm / base_resolution[1] / (2 ** annotation_resolution)
    pad_vz = box_radius_nm / base_resolution[2]
    xr = (center[0] - pad_vx, center[0] + pad_vx)
    yr = (center[1] - pad_vy, center[1] + pad_vy)
    zr = (center[2] - pad_vz, center[2] + pad_vz)

    for i in range(len(xyz)):
        x = xyz[i][0]
        y = xyz[i][1]
        z = xyz[i][2]

        if (
            x > xr[0]
            and x < xr[1]
            and y > yr[0]
            and y < yr[1]
            and z > zr[0]
            and z < zr[1]
        ):
            xyz_out.append(xyz[i])
            id_out.append(ids[i])

    return np.array(xyz_out), np.array(id_out, dtype=np.object)


def synapse_match(
    xyz_truth, xyz_detect, id_truth, id_detect, thresh
):  # pylint: disable=R0914
    """
    <Description here>
    Parameters
    ----------
    xyz_truth : array_like
        <description>
    xyz_detect : array_like
        <description>
    id_truth : array_like
        <description>
    id_detech : array_like
        <description>
    thresh : float
        <description>
    Returns
    -------
    id_lookup : <type>
        <description>
    """

    # pylint: disable=C0103

    # Ensure we have numpy arrays
    xyz_truth = np.asarray(xyz_truth)
    xyz_detect = np.asarray(xyz_detect)
    id_truth = np.asarray(id_truth)
    id_detect = np.asarray(id_detect)

    cost, row_ind, col_ind = munkres_assignment(xyz_truth, xyz_detect)
    print(cost)
    match_idx = np.where(cost[row_ind, col_ind] < thresh)

    if len(match_idx) > 0:
        # row is idx of GT TP
        # col is idx of student TP
        gt_tp_idx, det_tp_idx = row_ind[match_idx], col_ind[match_idx]
        gt_tp_ids, det_tp_ids = id_truth[gt_tp_idx], id_detect[det_tp_idx]
        # Combine into pairs
        id_lookup = np.column_stack((gt_tp_ids, det_tp_ids))
    else:
        gt_tp_idx = []
        det_tp_idx = []
        id_lookup = np.column_stack(
            (np.array([], dtype="object"), np.array([], dtype="object"))
        )

    # not in row (set diff) are FN
    gt_syn_idx = np.arange(0, len(xyz_truth))
    fn_idx = np.setdiff1d(gt_syn_idx, gt_tp_idx)
    if len(fn_idx) > 0:
        fn_ids = id_truth[fn_idx]
        id_lookup_fn = np.column_stack((fn_ids, np.repeat(None, len(fn_ids))))
        id_lookup = np.concatenate((id_lookup, id_lookup_fn))

    # not in col (set diff) are FP
    det_syn_idx = set(np.arange(0, len(xyz_detect)))
    fp_idx = np.asarray(list(det_syn_idx.difference(det_tp_idx)))
    if len(fp_idx) > 0:
        fp_ids = id_detect[fp_idx]
        id_lookup_fp = np.column_stack((np.repeat(None, len(fp_ids)), fp_ids))
        id_lookup = np.concatenate((id_lookup, id_lookup_fp))

    return pd.DataFrame(id_lookup, columns=["ground_truth", "detect"])


def run_synapse_analysis(
    gt_xyzs,
    gt_ids,
    test_xyzs,
    test_ids,
    threshold,
    iso_corrected=False,
    iso_correction=1,
):
    """
    <Description here>
    Parameters
    ----------
    gt_xyzs : numpy.ndarray
        Array of ground truth xyz coordinates
    gt_ids : numpy.ndarray
        Array of ids associated with ground truth xyz coordinates
    test_xyzs : numpy.ndarray
        Array of test xyz coordinates
    test_ids : numpy.ndarray
        Array of ids associated with test xyz coordinates
    threshold : float
        Synapse matching threshold
    iso_corrected : boolean
        Mark whether the data is isotropic. If not, it will be made isotropic using the
        `iso_correction` parameter.
    iso_correction : float
        Value to correct anistropy.
    Returns
    -------
    sm : SynapseMetrics
        Resultant object containing precision, recall, and F1 scores, along with
        with true positive (both ground truth and test), false positive, and false negative
        ids.
    """
    # pylint: disable=R0913,R0914
    if not iso_corrected:
        # gt_xyzs = utils.make_isotropic(gt_xyzs, iso_correction)
        gt_xyzs = make_isotropic(gt_xyzs, iso_correction)
        # test_xyzs = utils.make_isotropic(test_xyzs, iso_correction)
        test_xyzs = make_isotropic(test_xyzs, iso_correction)

    results_table = synapse_match(
        gt_xyzs, test_xyzs, gt_ids, test_ids, threshold)

    tp = results_table.dropna()
    fn = results_table[results_table.detect.isnull()]
    fp = results_table[results_table.ground_truth.isnull()]

    assert len(tp.ground_truth) == len(
        tp.detect
    ), "true positive ground truth and test size mismatch"

    tp_count = len(tp)
    fp_count = len(fp)
    fn_count = len(fn)

    try:
        # precision, recall = utils.calculate_precision_recall(
        #     tp_count, fp_count, fn_count
        # )

        precision, recall = calculate_precision_recall(
            tp_count, fp_count, fn_count
        )

    except ZeroDivisionError:
        precision, recall = np.nan, np.nan

    try:
        # f1 = utils.calculate_f1(precision, recall)
        f1 = calculate_f1(precision, recall)
    except ZeroDivisionError:
        f1 = np.nan

    sm = SynapseMetrics(
        precision=precision,
        recall=recall,
        f1=f1,
        tp_gt_ids=np.asarray(tp.ground_truth),
        tp_test_ids=np.asarray(tp.detect),
        fp_ids=np.asarray(fp.detect),
        fn_ids=np.asarray(fn.ground_truth),
    )
    return sm


### Running metrics

In [12]:
# Loading in data
load = pd.read_csv("agents/Data/endpoints_gt5.csv")
# endpoints_dict = {}

# #creating series objects for the desired collumns
# segIDs = load['seg_id']
# Endpoints = load['endpoints']

In [13]:
# Generate out_df

out_df = pd.DataFrame(columns=["run", "precision", "recall", "f1", "seg_id",
                      "num_endpoints", "endpoints", "comments", "proposed_endpoints"])


In [14]:
run = 0
# for invalidation_d in range(4000, 6001, 200):
for invalidation_d in range(4000, 4201, 200):
    # for num_humfrey_iters in range(0,251,25):
    for num_humfrey_iters in range(50, 76, 25):
        # for decimation_factor in np.arange(0.3, 0.71, 0.04):
        for decimation_factor in np.arange(0.3, 0.35, 0.04):
            load = pd.read_csv("agents/Data/endpoints_gt5.csv") # NOT THE BEST WAY TO DO THIS!!!
            run += 1
            load_with_proposed_endpoints = endpoint_generator(
                load, invalidation_d=invalidation_d, humfrey_iters=num_humfrey_iters, decimation_factor=decimation_factor)
            out_df = testing_metrics(
                load_with_proposed_endpoints, threshold=500, run=run, output=out_df)

07/28/2022 01:10:10 PM WARNING: face_normals incorrect shape, ignoring! 


100%|██████████| 3996/3996 [00:00<00:00, 436481.13it/s]
07/28/2022 01:10:13 PM WARNING: face_normals incorrect shape, ignoring! 


100%|██████████| 10649/10649 [00:00<00:00, 321519.33it/s]
07/28/2022 01:10:16 PM WARNING: face_normals incorrect shape, ignoring! 


100%|██████████| 1140/1140 [00:00<00:00, 256821.71it/s]
07/28/2022 01:10:21 PM WARNING: face_normals incorrect shape, ignoring! 


100%|██████████| 91376/91376 [00:00<00:00, 143297.59it/s]
07/28/2022 01:10:32 PM WARNING: face_normals incorrect shape, ignoring! 


...

07/28/2022 01:17:56 PM WARNING: face_normals incorrect shape, ignoring! 




Seg 864691133712131152 returned error on get. Skipping.



07/28/2022 01:17:58 PM WARNING: face_normals incorrect shape, ignoring! 


100%|██████████| 204/204 [00:00<00:00, 89821.33it/s]

[[4423.90008929   99.46356117]
 [  24.37211521 4415.78452826]]
[[4435.7557417  9272.57704201 5470.51569781  114.09206809]
 [1304.39602882 8350.73439884  495.77414213 5522.29273038]
 [7402.44311022  368.27978495 8664.4402589  9209.50161518]]
skipping 

skipping 

[[226.46191733 173.70377083]
 [252.88337233 146.63560277]]
[[495.25548962 358.60842154]
 [233.07724042 491.02851241]]
[[2869.29137593   76.92853827 4494.89933146]
 [1637.34693941 4502.33339503  624.97999968]]
[[517.58284361 901.23748258]
 [572.19926599 826.68313156]]
skipping 

[[ 81.63332653 648.0686692 ]
 [605.6153895  107.42904635]]
[[  114.63420083  4128.18483113  3189.93291466]
 [ 4026.12592948   170.76592166  3482.68416598]
 [11962.48092997 11292.02129824  8929.37315829]
 [16572.55924714 13982.45489891 13733.27073206]
 [ 6085.85096761  7099.01887587  3574.97762231]]
[[7194.75399162 8128.64742746  864.88033854]
 [1455.19070915  315.13171849 8280.50898194]]
[[ 3717.38079836    85.11756575 10550.35264813]
 [ 7734.51601589 10

In [1]:
out_df.to_csv('results.csv')

NameError: name 'out_df' is not defined